In [ ]:
# user managed notebook = TensorFlow Enterprise 2.6 withou GPU
# install tfdv running this in terminal: pip install --user tensorflow-data-validation
# if data lives in GCS: pip install --user 'google.cloud.storage==1.44.0'

In [ ]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
from tensorflow_data_validation.utils import slicing_util

import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from google.cloud import storage

In [2]:
#####################################################################
#
# if your data is in GCS....
#
#####################################################################

In [3]:
# set params
PROJECT_ID = "my-project"
GCS_BUCKET = "my-bucket"
REGION = "us-central1"

# get training data
gcs_client = storage.Client(project=PROJECT_ID)
bucket = gcs_client.get_bucket(GCS_BUCKET)
blob_name = "tabular_binary_class_even_split_slim.csv"
blob = bucket.blob(blob_name)
blob.download_to_filename(blob_name)

print("Downloaded blob {} to {}.".format(blob.name, blob_name))

Downloaded blob tabular_binary_class_even_split_slim.csv to tabular_binary_class_even_split_slim.csv.


In [ ]:
#####################################################################
#
# for this exercise, generate synthetic data using sklearn
#
#####################################################################

In [2]:
# create "training" data
x, y = make_classification(n_classes=2
                           , n_samples=10000
                           , n_features=5
                           , n_informative=5
                           , n_redundant=0
                           , n_repeated=0
                           , random_state=34098)

training_data = np.concatenate((x, y[:,None]), axis=1)
num_cols = x.shape[1]
col_names = ','.join([f"x_{i}" for i in range(num_cols)] + ['label'])
col_names

'x_0,x_1,x_2,x_3,x_4,label'

In [3]:
# save training data to csv
train_data_file = 'train.csv'
np.savetxt( train_data_file
           , training_data
           , delimiter=','
           , fmt='%f'
           , header=col_names
           , comments="")

In [4]:
# generate synthetic eval data from a totally diff distribution
eval_data_file = 'eval.csv'

eval_data = np.random.rand(10000,5)+100

num_cols = eval_data.shape[1]
col_names = ','.join([f"x_{i}" for i in range(num_cols)])
print(col_names)

np.savetxt(  eval_data_file
           , eval_data
           , delimiter=','
           , fmt='%f'
           , header=col_names
           , comments="")

x_0,x_1,x_2,x_3,x_4


In [5]:
TRAIN_DATA = train_data_file
EVAL_DATA = eval_data_file

In [ ]:
train_stats = tfdv.generate_statistics_from_csv(data_location=TRAIN_DATA)

In [7]:
tfdv.visualize_statistics(train_stats)

In [8]:
# generate schema from training data
schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'x_0',FLOAT,required,,-
'x_1',FLOAT,required,,-
'x_2',FLOAT,required,,-
'x_3',FLOAT,required,,-
'x_4',FLOAT,required,,-
'label',FLOAT,required,,-


In [9]:
# Compute stats for evaluation data
eval_stats = tfdv.generate_statistics_from_csv(data_location=EVAL_DATA)

In [10]:
# Compare evaluation data with training data
tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

In [ ]:
#####################################################################
#
#  Schema validation
#
#####################################################################

In [11]:
# Check eval data for errors by validating against previously inferred schema.
anomalies = tfdv.validate_statistics(statistics=eval_stats, schema=schema)
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'label',Column dropped,Column is completely missing


In [ ]:
#####################################################################
#
#  Jensen-Shannon divergence: Drift
#
#####################################################################

In [12]:
# with no comparators set, no anomalies are detected
skew_anomalies = tfdv.validate_statistics(train_stats, schema, previous_statistics=eval_stats)
tfdv.display_anomalies(skew_anomalies)

In [13]:
# set comparators for drift (current versus previous)
x_0 = tfdv.get_feature(schema, 'x_0')
x_0.drift_comparator.jensen_shannon_divergence.threshold = 0.001

#note: previous_statistics in next line relates to drift_comparator above
skew_anomalies = tfdv.validate_statistics(train_stats, schema, previous_statistics=eval_stats)
tfdv.display_anomalies(skew_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'x_0',High approximate Jensen-Shannon divergence between current and previous,"The approximate Jensen-Shannon divergence between current and previous is 1 (up to six significant digits), above the threshold 0.001."


In [14]:
# set comparators for drift (current versus previous)
x_0 = tfdv.get_feature(schema, 'x_0')
x_0.drift_comparator.jensen_shannon_divergence.threshold = 1.0

x_1 = tfdv.get_feature(schema, 'x_1')
x_1.drift_comparator.jensen_shannon_divergence.threshold = 0.1

x_2 = tfdv.get_feature(schema, 'x_2')
x_2.drift_comparator.jensen_shannon_divergence.threshold = 0.001

x_3 = tfdv.get_feature(schema, 'x_3')
x_3.drift_comparator.jensen_shannon_divergence.threshold = 0.0001

x_4 = tfdv.get_feature(schema, 'x_4')
x_4.drift_comparator.jensen_shannon_divergence.threshold = 0.00001

skew_anomalies = tfdv.validate_statistics(train_stats, schema, previous_statistics=eval_stats)
tfdv.display_anomalies(skew_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'x_4',High approximate Jensen-Shannon divergence between current and previous,"The approximate Jensen-Shannon divergence between current and previous is 1 (up to six significant digits), above the threshold 1e-05."
'x_2',High approximate Jensen-Shannon divergence between current and previous,"The approximate Jensen-Shannon divergence between current and previous is 1 (up to six significant digits), above the threshold 0.001."
'x_3',High approximate Jensen-Shannon divergence between current and previous,"The approximate Jensen-Shannon divergence between current and previous is 1 (up to six significant digits), above the threshold 0.0001."
'x_1',High approximate Jensen-Shannon divergence between current and previous,"The approximate Jensen-Shannon divergence between current and previous is 1 (up to six significant digits), above the threshold 0.1."


In [ ]:
#####################################################################
#
#  Jensen-Shannon divergence: Skew
#
#####################################################################

In [16]:
# set comparators for skew (training versus serving)
x_0 = tfdv.get_feature(schema, 'x_0')
x_0.skew_comparator.jensen_shannon_divergence.threshold = 0.001

#note: serving_statistics in next line relates to skew_comparator above
skew_anomalies = tfdv.validate_statistics(train_stats, schema, serving_statistics=eval_stats)
tfdv.display_anomalies(skew_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'x_0',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 1 (up to six significant digits), above the threshold 0.001."


In [17]:
# set comparators for skew (training versus serving)
x_0 = tfdv.get_feature(schema, 'x_0')
x_0.skew_comparator.jensen_shannon_divergence.threshold = 1.0

x_1 = tfdv.get_feature(schema, 'x_1')
x_1.skew_comparator.jensen_shannon_divergence.threshold = 0.1

x_2 = tfdv.get_feature(schema, 'x_2')
x_2.skew_comparator.jensen_shannon_divergence.threshold = 0.001

x_3 = tfdv.get_feature(schema, 'x_3')
x_3.skew_comparator.jensen_shannon_divergence.threshold = 0.0001

x_4 = tfdv.get_feature(schema, 'x_4')
x_4.skew_comparator.jensen_shannon_divergence.threshold = 0.00001

#note: serving_statistics in next line relates to skew_comparator above
skew_anomalies = tfdv.validate_statistics(train_stats, schema, serving_statistics=eval_stats)
tfdv.display_anomalies(skew_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'x_1',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 1 (up to six significant digits), above the threshold 0.1."
'x_3',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 1 (up to six significant digits), above the threshold 0.0001."
'x_2',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 1 (up to six significant digits), above the threshold 0.001."
'x_4',High approximate Jensen-Shannon divergence between training and serving,"The approximate Jensen-Shannon divergence between training and serving is 1 (up to six significant digits), above the threshold 1e-05."
